In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


**FOR BODY DAMAGE**

**IMAGE PRE PROCESSING**

**1. Import The ImageDataGenerator Library**

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

**2. Configure ImageDataGenerator Class**

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.1,
                                   zoom_range = 0.1,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

**3. Apply ImageDataGenerator Functionality To Trainset And Testset**

In [4]:
training_set = train_datagen.flow_from_directory('/content/gdrive/MyDrive/Car damage/body/training',
                                                 target_size = (224, 224),
                                                 batch_size = 10,
                                                 class_mode = 'categorical')
test_set = test_datagen.flow_from_directory('/content/gdrive/MyDrive/Car damage/body/validation',
                                            target_size = (224, 224),
                                            batch_size = 10,
                                            class_mode = 'categorical')

Found 979 images belonging to 3 classes.
Found 171 images belonging to 3 classes.


**MODEL BUILDING**

**1. Importing The Model Building Libraries**

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

**2. Loading The Model**

In [6]:
IMAGE_SIZE = [224, 224]

train_path = '/content/gdrive/MyDrive/Car damage/body/training'
valid_path = '/content/gdrive/MyDrive/Car damage/body/validation'

In [7]:
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58889256/58889256 [==============================] - 0s 0us/step


**3. Adding Flatten Layer**

In [8]:
for layer in vgg16.layers:
    layer.trainable = False

In [9]:
folders = glob('/content/gdrive/MyDrive/Car damage/body/training/*')

In [10]:
folders

['/content/gdrive/MyDrive/Car damage/body/training/02-side',
 '/content/gdrive/MyDrive/Car damage/body/training/01-rear',
 '/content/gdrive/MyDrive/Car damage/body/training/00-front']

In [11]:
x = Flatten()(vgg16.output)

In [12]:
len(folders)

3

**4. Adding Output Layer**

In [13]:
prediction = Dense(len(folders), activation='softmax')(x)

**5. Creating A Model Object**

In [14]:
model = Model(inputs=vgg16.input, outputs=prediction)

In [15]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

**6. Configure The Learning Process**

In [16]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

**7. Train The Model**

In [17]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=25,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/25
98/98 [==============================] - 645s 7s/step - loss: 1.2343 - accuracy: 0.5128 - val_loss: 0.8886 - val_accuracy: 0.6491
Epoch 2/25
98/98 [==============================] - 633s 6s/step - loss: 0.6810 - accuracy: 0.7487 - val_loss: 0.8953 - val_accuracy: 0.6667
Epoch 3/25
98/98 [==============================] - 636s 6s/step - loss: 0.5376 - accuracy: 0.7967 - val_loss: 0.9190 - val_accuracy: 0.6374
Epoch 4/25
98/98 [==============================] - 635s 6s/step - loss: 0.4057 - accuracy: 0.8458 - val_loss: 1.2112 - val_accuracy: 0.6199
Epoch 5/25
98/98 [==============================] - 639s 7s/step - loss: 0.3807 - accuracy: 0.8570 - val_loss: 1.4361 - val_accuracy: 0.6608
Epoch 6/25
98/98 [==============================] - 635s 6s/step - loss: 0.2567 - accuracy: 0.9101 - val_loss: 1.0055 - val_accuracy: 0.6550
Epoch 7/25
98/98 [==============================] - 637s 6s/step - loss: 0.1830 - accuracy: 0.9367 - val_loss: 1.2190 - val_accuracy: 0.6667
Epoch 8/25
98

**8. Save The Model**

In [18]:
from tensorflow.keras.models import load_model

model.save('/content/gdrive/MyDrive/Car damage/body.h5')

**9. Test The Model**

In [19]:
from tensorflow.keras.models import load_model
import cv2
from skimage.transform import resize

In [20]:
model = load_model('/content/gdrive/MyDrive/Car damage/body.h5')

In [21]:
def detect(frame):
  img = cv2.resize(frame,(224,224))
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

  if(np.max(img)>1):
    img = img/255.0
  img = np.array([img])
  prediction = model.predict(img)
  label = ["front","rear","side"]
  preds = label[np.argmax(prediction)]
  return preds

In [22]:
import numpy as np

In [23]:
data = "/content/gdrive/MyDrive/Car damage/body/training/00-front/0010.JPEG"
image = cv2.imread(data)
print(detect(image))

1/1 [==============================] - 1s 716ms/step
front
